In [1]:

import pandas as pd
import json
import re

# Función para limpiar strings para UIDs
def clean_str(s):
    return re.sub(r'\W+', '_', str(s))

# Cargar los datos
df_comm = pd.read_csv("kiali_kpi_metrics.csv")

# Filtrar registros válidos
df_comm['time_window'] = df_comm['time_window'].astype(str).str.strip()
df_comm = df_comm[df_comm['time_window'] == "15S"].copy()
df_comm['error_rate'] = pd.to_numeric(df_comm['error_rate'], errors='coerce').fillna(0.0)
df_comm = df_comm[df_comm['istio_request_bytes'] != 0.0]
df_comm = df_comm[df_comm['new_request'] >= 0.0]
df_comm = df_comm.dropna(subset=["request_rate"])

# Asignar status
def assign_status(er):
    return 0 if er < 0.15 else 1
df_comm['status'] = df_comm['error_rate'].apply(assign_status)

# Crear nodos microservicio
microservices = set(df_comm['source_workload']).union(set(df_comm['destination_workload']))
nodes = []
node_uid_map = {}
for ms in microservices:
    uid = f"_:ms_{clean_str(ms)}"
    nodes.append({
        "uid": uid,
        "dgraph.type": "Microservice",
        "microservice": {
            "name": ms
        }
    })
    node_uid_map[ms] = uid

# Crear relaciones de comunicación
edges = []
for _, row in df_comm.iterrows():
    src = node_uid_map[row["source_workload"]]
    tgt = node_uid_map[row["destination_workload"]]
    edge = {
        "uid": f"_:edge_{clean_str(row['source_workload'])}_{clean_str(row['destination_workload'])}_{clean_str(str(row['timestamp']))}",
        "dgraph.type": "Communication",
        "from": {"uid": src},
        "to": {"uid": tgt},
        "communication": {
            "timestamp": str(row["timestamp"]),
            "throughput": row["throughput"],
            "duration_milliseconds": row["duration_milliseconds"],
            "request_rate": row["request_rate"],
            "istio_request_bytes": row["istio_request_bytes"],
            "average_latency": row["average_latency"],
            "new_request": row["new_request"],
            "status": int(row["status"])
        }
    }
    edges.append(edge)

# Guardar grafo completo
graph_comm = nodes + edges
with open("communication_graph_cleaned.json", "w", encoding="utf-8") as f:
    json.dump(graph_comm, f, indent=2)

print("✅ communication_graph_cleaned.json creado correctamente.")


✅ communication_graph_cleaned.json creado correctamente.
